In [ ]:
from copy import deepcopy
import time
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import json
from activity_detector import ActivityDetector, LateralActivityHost, LateralActivityTarget, \
    LeadVehicle, ActivityDetectorParameters, LineData, LateralStateTarget, LongitudinalStateTarget, \
    LongitudinalActivity
from mark_highway import HighwayMarker
from find_index_from_video import approx_index
from ngram import NGram
from ngram_search import find_sequence
import seaborn as sns
from tqdm import tqdm
from typing import Iterable, List, NamedTuple, Tuple
from target_gluer import merge_targets, _TargetGluer, _TargetGluerOptions
%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [ ]:
# Load the data
i_file = 0
datafiles = glob(os.path.join("data", "1_hdf5", '*.hdf5'))
AD = ActivityDetector(datafiles[i_file])

In [ ]:
AD.targets[0].keys()

In [ ]:
AD.set_lon_activities_host()

In [ ]:
AD.set_lat_activities_host()

The following codes checks the recall of the lane changed detection. It shows the activity at the index at which a lane change occurs. These indices are obtained after looking at the video. 

It shows that all left lane changes are detected. Two right lane changes are missed. 

In [ ]:
if i_file == 0:
    # Jeroen's code does not detect the lane change at 1963.60.
    i_left_lane_change = [203.35, 217.29, 1738.83, 1813.3, 1900.71, 1963.60,
                          2000.3, 2171.98, 2310.65, 2318.23, 2546.01, 2650.02]
    print([AD.get("host_lateral_activity", i) for i in i_left_lane_change])
    # Jeroen's code does not detect the lane change at 2193.44 and 2236.72.
    # These lane changes are also difficult to see by eye, so these indices
    # are approximate.
    i_right_lane_change = [251.46, 304.95, 1846.39, 1858.69, 1927.05, 2040.03,
                           2193.44, 2236.72, 2487.13, 2568.65, 2668.15, 2675.00]
    print([AD.get("host_lateral_activity", i) for i in i_right_lane_change])

The following code shows the recall of the cut-in detection at the highway. There are six cut-ins identified from the videos. The lateral activities for these six target vehicles are determined. 
All cut-ins are detected. For some strange reason, the lane changes are detected 3 seconds before the timing that is obtained from the video. That might be a syncing error?

In [ ]:
if i_file == 0:
    # Cut-in targets:
    # 3850: Motorbike from right (29:47, index=1802)
    # 4171: Dark passenger car from right (quite far) (32:50, index=1985)
    # 5028: Silver passenger car from right (39:18, index=2373)
    # 5191: Black van from left (40:53, index=2468)
    # 5645: Black passenger car from left (42:26, index=2562)
    # 6008: Gray VW from left (44:26, index=2681)
    for i_target, index in zip([3850, 4171, 5028, 5191, 5645, 6008],
                               [1802., 1985., 2373., 2468., 2562., 2681.]):
        AD.set_target_activities(i_target)
        print("Activity of target {:d} at index {:.2f}: {:s}".
              format(i_target, index-3, AD.targets[i_target].at[index-3, "lateral_activity"]))

Compute all events for the targets. This takes quite some time, around 13 minutes on the zBook laptop.

In [ ]:
# This takes round 13 minutes!
AD.set_target_activities()

Set the tags for the longitudinal and lateral state of each target vehicle at each time instant. Furthermore, tags are added that specify whether a target vehicle is a lead vehicle at a certain time instant. The states of the targets (i.e., `set_states_targets()`) can only be done after the lateral events (through `set_target_activities()`), because the lateral position of the targets with respect to the lane lines is needed. Similarly, the determination of the lead vehicle (`set_lead_vehicle()`) can only be done after the other states (`set_states_targets()`), because it makes use of the result of the previous step.

`set_states_targets()` is really fast, `set_lead_vehicle()` takes around 3 minutes on the zBook laptop.

In [ ]:
AD.set_states_targets()
AD.set_lead_vehicle()

Determine whether the ego vehicle drives on the highway.

In [ ]:
highway_marker = HighwayMarker(AD.data)
highway_marker.mark_highway()

Save the data!

In [ ]:
AD.to_hdf(datafiles[i_file])

## Create or load n-grams for target vehicles

In [ ]:
filename = os.path.join("data", "4_ngrams", "{:s}.hdf5".
                        format(os.path.splitext(os.path.basename(datafiles[i_file]))[0]))
fieldnames = ["longitudinal_activity", "lateral_activity", "longitudinal_state", 
              "lateral_state", "lead_vehicle", "id"]
metadata = (("tstart", float), ("tend", float), ("target_id", int))
target_ngrams = NGram(fieldnames, metadata)

In [ ]:
if True or not target_ngrams.from_hdf(filename, "targets"):
    tstart = time.time()
    for target in AD.targets:
        target_ngrams.ngram_from_data(target, 
                                      tstart=target.index[0],
                                      tend=target.index[-1],
                                      target_id=int(target["id"].values[0]))
    target_ngrams.sort_ngrams("tstart")
    target_ngrams.to_hdf(filename, "targets", mode="w")
    print("Total elapsed time: {:.2f} s".format(time.time() - tstart))

## Create or load n-gram for ego vehicle

In [ ]:
fieldnames = ["host_longitudinal_activity", "host_lateral_activity", "is_highway"]
metadata = (("tstart", float), ("tend", float))
ego_ngram = NGram(fieldnames, metadata)

In [ ]:
if True or not ego_ngram.from_hdf(filename, "ego"):
    tstart = time.time()
    ego_ngram.ngram_from_data(AD.data, tstart=AD.data.index[0], tend=AD.data.index[-1])
    ego_ngram.to_hdf(filename, "ego")
    print("Total elapsed time: {:.2f} s".format(time.time() - tstart))

## Extract cut-in scenario

For target vehicle, we need:
1. Lateral activity `li` or `ri`.
2. Lateral activity `fl` and lead vehicle `y`.

For the ego vehicle, we need it to go straight during both steps of the target vehicle. Furthermore, the scenario need to take place at the highway.

In [ ]:
target_tags = [dict(lateral_activity=[LateralActivityTarget.LEFT_CUT_IN.value, 
                                      LateralActivityTarget.RIGHT_CUT_IN.value],
                    lead_vehicle=[LeadVehicle.NOLEAD.value]),
               dict(lateral_activity=[LateralActivityTarget.LEFT_CUT_IN.value, 
                                      LateralActivityTarget.RIGHT_CUT_IN.value],
                    lead_vehicle=[LeadVehicle.LEAD.value])]
ego_tags = [dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True])]

In [ ]:
# Cut-in targets for first dataset:
# 3827: Motorbike from right (29:47, index=1799.29)
# 4099: Black SUV from left (32:28, index=1960.68)
# 4145: Dark passenger car from right (quite far) (32:50, index=1982.01)
# 4997: Silver passenger car from right (39:18, index=2370.28)
# 5155: Black van from left (40:53, index=2465.04)
# 5607: Black passenger car from left (42:26, index=2558.85)
# 5970: Gray VW from left (44:26, index=2677.92)
# File 2: [2411, 2415, 3264, 3435, 3729, (unknown), 4084, 4098]
# File 3: [4553, 5364, 5386, 5423, 5538]
# File 4: [723, (unknown), 3028, 3066, (unknown), 3419, 4197, 4228, 4272, 4520, 5020]
# File 5: [2299, 3932, 3942, 4713, 5025]
# File 6: [511, 2671, 2913, 3044, 4258, 4258]
tstart = time.time()
e = ego_ngram.ngram
cutins = []
for i, t in enumerate(target_ngrams.ngrams):
    search = find_sequence((t, e), (target_tags, ego_tags))
    while search.is_found:
        cutins.append((i, search.t_start, search.t_end))
        print(i, search)
        search = find_sequence((t, e), (target_tags, ego_tags), t_start=search.t_end+5)
print("Total elapsed time: {:.2f} s".format(time.time() - tstart))

Show the data at a cut in.

In [ ]:
maxy = 5
targets = pd.concat(AD.targets)
targets = targets.loc[np.logical_and(targets['dx'] > 0, targets['dx'] < 80)]
targets = targets.loc[np.logical_and(targets['dy'] > -maxy, targets['dy'] < maxy)]
ids = [target["id"].values[0] for target in AD.targets]

In [ ]:
def plot_cutin():
    index_approx = approx_index(minutes, seconds, AD.data)
    t_start = index_approx - window
    t_end = index_approx + window
    
    targets_update = targets.loc[np.logical_and(targets.index >= t_start, targets.index <= t_end)]
    plt.plot([t_start, t_end], [0, 0], 'k-')
    for target_id, target in targets_update.groupby("id"):
        plt.plot(target["line_left_next"], 'b-')
        plt.plot(target["line_right_next"], 'r-')
        
        # Try to find cut-in
        potential = np.logical_or(np.logical_and(target["line_left_next"] > 0,
                                                 target["line_left_prev"] < 0),
                                  np.logical_and(target["line_right_next"] < 0,
                                                 target["line_right_prev"] > 0))
        if np.any(potential):
            print("Potential cut-in of target ID {:.0f} (=target {:d}) at: "
                  .format(target_id, ids.index(target_id)), end="")
            i = target.index[potential].values[0]
            print("{:.2f} (dx={:.1f}, thw={:.2f})"
                  .format(i, target.loc[i, "dx"], target.loc[i, "dx"]/AD.data.loc[i, "Host_vx"]))
    plt.xlim(t_start, t_end)
    plt.ylim(-maxy, maxy)

In [ ]:
minutes = 42
seconds = 16
window = 5
plot_cutin()

In [ ]:
i_target = 2395
target_ngrams.ngrams[i_target]

In [ ]:
ego_ngram.ngram[target_ngrams.metadata["tstart"].values[i_target]-10:
                target_ngrams.metadata["tend"].values[i_target]]

In [ ]:
find_sequence((target_ngrams.ngrams[i_target], ego_ngram.ngram), 
              (target_tags, ego_tags), verbose=2)

In [ ]:
[print(event) for event in AD.lat_activities_target_i(i_target)]
AD.set_target_activities(i_target)
plt.plot(AD.targets[i_target]["line_left_next"], 'b')
plt.plot(AD.targets[i_target]["line_right_next"], 'r')
plt.plot([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]], [0, 0], 'k')
plt.plot(AD.targets[i_target]["line_left_down"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_left_up"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_right_down"], color=(.5, .5, 1))
plt.plot(AD.targets[i_target]["line_right_up"], color=(.5, .5, 1))
_ = plt.xlim([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]])

In [ ]:
i = 2430.48
AD.targets[i_target].loc[i, "dx"] / AD.data.loc[i, "Host_vx"]

In [ ]:
[print(event) for event in AD.lat_activities_target_i(i_target)]
#AD.set_target_activities(i_target)
plt.plot(AD.targets[i_target]["line_left_next"], 'b')
plt.plot(AD.targets[i_target]["line_right_next"], 'r')
plt.plot([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]], [0, 0], 'k')
plt.plot(AD.targets[i_target]["line_left_down"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_left_up"], color=(1, .5, .5))
plt.plot(AD.targets[i_target]["line_right_down"], color=(.5, .5, 1))
plt.plot(AD.targets[i_target]["line_right_up"], color=(.5, .5, 1))
_ = plt.xlim([AD.targets[i_target].index[0], AD.targets[i_target].index[-1]])

## Extract host lane changes

In [ ]:
def plot_lane_change():
    index_approx = approx_index(minutes, seconds, AD.data)
    t_start = index_approx - window
    t_end = index_approx + window
    
    plt.plot(AD.data.loc[t_start:t_end, "lines_0_c0_prev"])
    plt.plot(AD.data.loc[t_start:t_end, "lines_1_c0_prev"])
    potentials = np.logical_and(np.abs(AD.data.loc[t_start:t_end, "lines_0_c0_diff"]) > 1,
                                np.abs(AD.data.loc[t_start:t_end, "lines_1_c0_diff"]) > 1)
    for i in np.where(potentials)[0]:
        print("Potential lane change at {:.2f}".format(potentials.index[i]))

In [ ]:
ego_tags = [dict(host_lateral_activity=[LateralActivityHost.LEFT_LANE_CHANGE.value],
                 is_highway=[True])]
search = find_sequence((ego_ngram.ngram,), (ego_tags,))
while search.is_found:
    print("Left lane change from {:.2f} to {:.2f}".format(search.t_start, search.t_end))
    search = find_sequence((ego_ngram.ngram,), (ego_tags,), t_start=search.t_end)

In [ ]:
ego_tags = [dict(host_lateral_activity=[LateralActivityHost.RIGHT_LANE_CHANGE.value],
                 is_highway=[True])]
search = find_sequence((ego_ngram.ngram,), (ego_tags,))
while search.is_found:
    print("Right lane change from {:.2f} to {:.2f}".format(search.t_start, search.t_end))
    search = find_sequence((ego_ngram.ngram,), (ego_tags,), t_start=search.t_end)

In [ ]:
minutes = 42
seconds = 50
window = 10
plot_lane_change()

In [ ]:
plot_overtaking()

## Extract overtaking before changing lane

In [ ]:
target_tags = [dict(lateral_state=[LateralStateTarget.LEFT.value],
                    longitudinal_state=[LongitudinalStateTarget.REAR.value]),
               dict(lateral_state=[LateralStateTarget.LEFT.value],
                    longitudinal_state=[LongitudinalStateTarget.FRONT.value]),
               dict(lateral_state=[LateralStateTarget.LEFT.value],
                    longitudinal_state=[LongitudinalStateTarget.FRONT.value]),
               dict(lateral_state=[LateralStateTarget.SAME.value],
                    longitudinal_state=[LongitudinalStateTarget.FRONT.value])]
ego_tags = [dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LANE_FOLLOWING.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LEFT_LANE_CHANGE.value],
                 is_highway=[True]),
            dict(host_lateral_activity=[LateralActivityHost.LEFT_LANE_CHANGE.value],
                 is_highway=[True])]

In [ ]:
# File 1: [3836, 5429]
# File 2: [2278, 2616]
# File 3: [4400, 4588, 4661]
# File 4: [468, 484, 516, 4060, 4228, 4552]
# File 5: [466, 494, 3942]
# File 6: [4524, 4795]
e = ego_ngram.ngram
for i, t in enumerate(target_ngrams.ngrams):
    search = find_sequence((t, e), (target_tags, ego_tags))
    if search.is_found:
        print(i, search)

In [ ]:
maxy = 5
targets = pd.concat(AD.targets)
targets = targets.loc[np.abs(targets['dx']) < 100]
targets = targets.loc[np.logical_and(targets['dy'] > -1, targets['dy'] < maxy)]
ids = [target["id"].values[0] for target in AD.targets]

In [ ]:
def plot_overtaking():
    index_approx = approx_index(minutes, seconds, AD.data)
    t_start = index_approx - window
    t_end = index_approx + window
    
    targets_update = targets.loc[np.logical_and(targets.index >= t_start, targets.index <= t_end)]
    target_ids = []
    for target_id, target in targets_update.groupby("id"):
        # plt.plot(-target["dy"], target["dx"], '.')
        if np.any(target["lead_vehicle"] == "y"):
            print("Target {:d} is lead vehicle from {:.2f}"
                  .format(ids.index(target_id), 
                          target.index[np.where(target["lead_vehicle"] == "y")[0][0]]))
        i = ids.index(target_id)
        target_ids.append(i)
        search = find_sequence((target_ngrams.ngrams[i],), 
                               ([dict(longitudinal_state=['r']), dict(longitudinal_state=['f'])],))
        if search.is_found:
            print("Target {:d} goes from rear to front between {:.2f} and {:.2f}"
                  .format(i, search.t_start, search.t_end))
            plt.plot(-target["dy"], target["dx"], '.')
    print(target_ids)
    i_start = ego_ngram.ngram.index.get_loc(t_start, 'pad')
    i_end = ego_ngram.ngram.index.get_loc(t_end, 'backfill')
    print(ego_ngram.ngram.iloc[i_start:i_end+1])    

In [ ]:
minutes = AD.data.loc[234.59, "video_time"] / 10 / 60
seconds = (minutes - int(minutes)) * 60
print("{:.0f}:{:02.0f}".format(int(minutes), seconds))

In [ ]:
minutes = 3
seconds = 17
window = 10
plot_overtaking()

In [ ]:
i_target = 4562
plt.plot(-AD.targets[i_target]["dy"], AD.targets[i_target]["dx"], '.')

In [ ]:
target_ngrams.ngrams[i_target]

In [ ]:
find_sequence((target_ngrams.ngrams[i_target], e), (target_tags, ego_tags), verbose=2)

In [ ]:
for i in range(i_target-1, i_target-250, -1):
    if not np.all(AD.targets[i]["dx"] < 0):
        continue
    if AD.targets[i].index[-1] >= AD.targets[i_target].index[0]:
        continue
    if AD.targets[i]["dy"].values[-1] < 0:
        continue
    if AD.targets[i]["dx"].values[-1] < -10:
        continue
    if abs(AD.targets[i]["vx"].values[-1] - AD.targets[i_target]["vx"].values[0]) > 5:
        continue
    v_target = (AD.targets[i]["vx"].values[-1] + AD.targets[i_target]["vx"].values[0])/2
    v_host = AD.data.loc[AD.targets[i].index[-1]:AD.targets[i_target].index[0], "Host_vx"]
    t_gone = AD.targets[i_target].index[0] - AD.targets[i].index[-1]
    x_diff = (AD.targets[i]["dx"].values[-1] - AD.targets[i_target]["dx"].values[0] +
              v_target*t_gone - np.trapz(v_host)*0.01)
    print("{:4d}, dx={:.1f}".format(i, x_diff))

In [ ]:
a = deepcopy(AD)
t = _TargetGluer(a.data, a.targets, 0.01, _TargetGluerOptions(x_max_visible=25))

In [ ]:
i = 4361
info = t.target_in_blind_spot(AD.targets[i])
info

In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
for j in [i, i_target]:
    ax1.plot(-AD.targets[j]["dy"], AD.targets[j]["dx"], '.')
    ax2.plot(AD.targets[j]["vx"])
ax1.set_xlabel("y [m]")
ax2.set_xlabel("Time [s]")
ax1.set_ylabel("x [m]")
ax2.set_ylabel("Speed [m/s]")

In [ ]:
vt = (AD.targets[i]["vx"].iat[-1] + AD.targets[i_target]["vx"].iat[0]) / 2
t_gone = AD.targets[i_target].index[0] - AD.targets[i].index[-1]
print(AD.targets[i]["dx"].iat[-1] - AD.targets[i_target]["dx"].iat[0] +
      vt*t_gone - 
      np.sum(AD.data.loc[AD.targets[i].index[-1]:AD.targets[i_target].index[0], "Host_vx"]*0.01))

In [ ]:
candidate = AD.targets[i_target]
t_gone = candidate.index[0] - info.last_t
print(t_gone > t.options.t_max_gone)
print(t_gone <= 0 or candidate.index[-1] - candidate.index[0] <= t.options.t_min_available)
x_candidate = candidate["dx"].values[0]
print(info.is_front and x_candidate > 0,
      not info.is_front and x_candidate < 0,
      not t.options.x_min_visible < x_candidate < t.options.x_max_visible)
speed_start = candidate["vx"][:candidate.index[0] + t.options.tsec_v_avg]
speed_start = np.mean(speed_start[speed_start > 0])
x_absoffset = abs(info.last_x + t_gone*(info.vx_target+speed_start)/2 - x_candidate -
                  np.trapz(t.host.loc[info.last_t:candidate.index[0],
                                      t.options.fieldname_host_vx])*t.timestep)
print(x_absoffset > t.options.x_deviation, x_absoffset)
print(abs(speed_start - info.vx_target) > t.options.v_deviation)

In [ ]:
# Check for ego lane change
window = 20
index_approx = approx_index(minutes, seconds, AD.data)
t_start = index_approx - window
t_end = index_approx + window
plt.plot(AD.data.loc[t_start:t_end, "lines_0_c0_next"])
plt.plot(AD.data.loc[t_start:t_end, "lines_1_c0_next"])
plt.plot(AD.data.loc[t_start:t_end, "lines_0_c0_diff"])
plt.plot(AD.data.loc[t_start:t_end, "lines_0_quality"])

In [ ]:
#plt.plot(AD.data["lines_0_c0_diff"], AD.data["lines_1_c0_diff"], '.')
plt.plot(AD.data.loc[t_start:t_end, "lines_0_c0_diff"], 
         AD.data.loc[t_start:t_end, "lines_1_c0_diff"], '.')

In [ ]:
AD.data.keys()

In [ ]:
plt.plot(np.linspace(0,1,len(AD.data)), np.sort(AD.data["speed_inc"].values))
plt.xlim(0.9, 1)

## Extract lead braking that causes braking

In [ ]:
target_tags = [dict(lead_vehicle=LeadVehicle.LEAD.value,
                    longitudinal_activity=LongitudinalActivity.DECELERATING.value)]
ego_tags = [dict(is_highway=[True])]

In [ ]:
for i, t in enumerate(target_ngrams.ngrams):
    search = find_sequence((t, e), (target_tags, ego_tags))
    if search.is_found:
        # Find maximum deceleration.
        acceleration = AD.targets[i].loc[search.t_start:search.t_end, "ax"]
        max_deceleration = acceleration.min()
        print(i, search, "Maximum deceleration: {:.2f}".format(-max_deceleration))

In [ ]:
minutes = AD.data.loc[1829.81, "video_time"] / 10 / 60
seconds = (minutes - int(minutes)) * 60
print("{:.0f}:{:02.0f}".format(int(minutes), seconds))

In [ ]:
targets = pd.concat(AD.targets)
targets = targets.loc[targets["lead_vehicle"] == "y"]
ids = [target["id"].values[0] for target in AD.targets]

In [ ]:
def plot_braking():
    index_approx = approx_index(minutes, seconds, AD.data)
    t_start = index_approx - window
    t_end = index_approx + window
    
    targets_update = targets.loc[np.logical_and(targets.index >= t_start, targets.index <= t_end)]
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    for target_id, target in targets_update.groupby("id"):
        ax1.plot(target["ax"])
        ax2.plot(target["vx"])
    ylim1 = ax1.get_ylim()
    ylim2 = ax2.get_ylim()
    for target_id, target in targets_update.groupby("id"):
        print("Lead vehicle: {:d}".format(ids.index(target_id)))
        i_target = ids.index(target_id)
        lon_acc = ''
        last_i = 0
        for r in target_ngrams.ngrams[i_target].itertuples():
            if r.longitudinal_activity != lon_acc:
                if lon_acc == "d":
                    print("Braking at {:.2f}".format((last_i+r.Index)/2))
                lon_acc = r.longitudinal_activity
                color = 'r' if lon_acc == "d" else "g" if lon_acc == "a" else "b"
                ax1.plot([r.Index, r.Index], ylim1, color)
                ax2.plot([r.Index, r.Index], ylim2, color)
                last_i = r.Index
    ax1.set_xlim(t_start, t_end)
    ax2.set_xlim(t_start, t_end)
    ax1.set_ylim(ylim1)
    ax2.set_ylim(ylim2)

In [ ]:
minutes = 44
seconds = 27
window = 10
plot_braking()

In [ ]:
i_target = 597
target_ngrams.ngrams[i_target]